<h2 style='text-align: center;'>AI Lab 2 -  Capstone Project</h2>

<h3 style='text-align: center;'>Author - Aloy Banerjee</h3>
<h3 style='text-align: center;'>Roll No. CH22M503</h3>

**Part 1: Kafka**
> For this part, you need to complete the following tasks:

    1. Install and Set-up Kafka on the Google Colab Notebook
    2. Stream data from the given dataset and break the data into batches of 10 records that are written to Kafka, separated by a sleep time of 10 seconds until 100 records are written.
    3. Use the Kafka consumer to read every 5 seconds from the producer. (Note: The dataset you will use for this part is YELP-review.json.)

In [2]:
import os
from google.colab import drive

# Check if Google Drive is already mounted
if not os.path.exists('/content/drive'):
    # Mount Google Drive if it's not already mounted
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")



Google Drive is already mounted.


**Install Kafka and Zookeeper**

In [3]:
!curl -sSOL https://downloads.apache.org/kafka/3.5.0/kafka_2.12-3.5.0.tgz
!tar -xzf kafka_2.12-3.5.0.tgz

In [4]:
!echo "Starting ZooKeeper service..."
!./kafka_2.12-3.5.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.12-3.5.0/config/zookeeper.properties

!echo "Starting Kafka service..."
!./kafka_2.12-3.5.0/bin/kafka-server-start.sh -daemon ./kafka_2.12-3.5.0/config/server.properties

!echo "Waiting for 10 secs until Kafka and ZooKeeper services are up and running..."

!sleep 10

!ps -ef | grep kafka

Starting ZooKeeper service...
Starting Kafka service...
Waiting for 10 secs until Kafka and ZooKeeper services are up and running...
root        6944       1  0 16:51 ?        00:00:02 java -Xmx512M -Xms512M -server -XX:+UseG1GC -XX:MaxGCPauseMillis=20 -XX:InitiatingHeapOccupancyPercent=35 -XX:+ExplicitGCInvokesConcurrent -XX:MaxInlineLevel=15 -Djava.awt.headless=true -Xlog:gc*:file=/content/kafka_2.12-3.5.0/bin/../logs/zookeeper-gc.log:time,tags:filecount=10,filesize=100M -Dcom.sun.management.jmxremote -Dcom.sun.management.jmxremote.authenticate=false -Dcom.sun.management.jmxremote.ssl=false -Dkafka.logs.dir=/content/kafka_2.12-3.5.0/bin/../logs -Dlog4j.configuration=file:./kafka_2.12-3.5.0/bin/../config/log4j.properties -cp /content/kafka_2.12-3.5.0/bin/../libs/activation-1.1.1.jar:/content/kafka_2.12-3.5.0/bin/../libs/aopalliance-repackaged-2.6.1.jar:/content/kafka_2.12-3.5.0/bin/../libs/argparse4j-0.7.0.jar:/content/kafka_2.12-3.5.0/bin/../libs/audience-annotations-0.13.0.jar:/cont

**Run Kafka and Zookeeper in daemon mode on port 9092**

In [40]:
!./kafka_2.12-3.5.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --replication-factor 1 --partitions 1 --topic yelp_reviews

Created topic yelp_reviews.


**Create new topics in kafka**

In [41]:
!./kafka_2.12-3.5.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic yelp_reviews

Topic: yelp_reviews	TopicId: rD97CiaxRaecEZCSCw6S3g	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: yelp_reviews	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


**Install OpenJDK**

In [42]:
!echo "Installing OpenJDK 8 JDK..."
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Installing OpenJDK 8 JDK...


**Install Kafka's client**

In [43]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.7 MB/s eta 0:00:00


**Importing Library**

In [7]:
import sys
import time
import numpy as np
import pandas as pd
import random
from multiprocessing import Process
from pandas import Timestamp
from datetime import datetime
import threading
import json
from json import dumps,loads
from kafka import KafkaProducer, KafkaConsumer
from kafka.errors import KafkaError

**Common Variables**

In [5]:
# Define the Kafka topic
topic = "yelp_reviews"
# Define the google drive path
drive_link = '/content/drive/MyDrive/CapstoneProjectData/yelp_academic_dataset_review.json'


**Common Method**

In [8]:
def read_json_data(file_path, num_lines=None):
    """
    Reads data from a specified JSON file and converts it to a list of dictionaries.
    Each dictionary corresponds to a line in the JSON file. If num_lines is not specified,
    the function will attempt to read all lines in the file.

    Args:
        file_path (str): The path of the JSON file to read.
        num_lines (int, optional): The maximum number of lines to read from the file.
            Defaults to None, which means all lines will be read.

    Returns:
        list: A list of dictionaries representing the JSON data read from the file.

    Raises:
        IOError: If the file cannot be read.
        json.JSONDecodeError: If there is an error decoding the JSON data.
    """
    json_data = []
    try:
        with open(file_path, 'r') as f:
            for i, line in enumerate(f):
                if num_lines is not None and i >= num_lines:
                    break
                record = json.loads(line)
                json_data.append(record)
    except IOError:
        raise IOError(f"Could not read file at path: {file_path}")
    except json.JSONDecodeError:
        raise json.JSONDecodeError(f"Error decoding JSON data in file: {file_path}")

    return json_data

def send_to_kafka(producer, topic, data, iteration):
    """
    Sends data to Kafka topic using the specified producer.

    Parameters:
        producer (KafkaProducer): The Kafka producer instance.
        topic (str): The name of the Kafka topic to send data to.
        data (list): A list of dictionaries representing the data to send.
        iteration(int) : Iteration Number

    Returns:
        None
    """
    try:
        print(f'yelp data length : {len(data)}')
        for record in data:
            if 'timestamp' in record and isinstance(record['timestamp'], str):
                record['timestamp'] = datetime.strptime(record['timestamp'], "%Y-%m-%d %H:%M:%S")
            message = json.dumps(record).encode("utf-8")
            print(message)
            producer.send(topic, value=message)
        print(f"Iteration:{iteration} data sent successfully to Kafka.")
    except KafkaError as e:
        print(f"Error sending data to Kafka: {e}")

def consume_messages(consumer):
    """
    Consumes messages from Kafka in a separate thread.

    Parameters:
        consumer (KafkaConsumer): The Kafka consumer instance.

    Returns:
        None
    """
    try:
        while True:  # Run indefinitely to keep consuming messages
            for message in consumer:
                record = message.value
                print(record)
                # Perform further processing of the record here if needed
            time.sleep(5)  # Sleep for 5 seconds before reading more messages
    except Exception as e:
        print(f"Error occurred while consuming messages: {e}")
    finally:
        consumer.close()

def kafka_producer_call(drive_link, topic):
    """
    Establishes a Kafka producer, loads Yelp data, and sends it to Kafka in batches.

    Args:
        drive_link (str): Path to the Yelp data JSON file.
        topic (str): Kafka topic to which data will be sent.
    """
    # Create a Kafka producer
    producer = KafkaProducer(bootstrap_servers="localhost:9092")
    print('Producer sending data:')
    try:
        # Load the Yelp data and select randomly 1000 records for sending to Kafka
        yelp_reviews_data = read_json_data(drive_link, num_lines=1000)
        num_lines_to_select = 100
        yelp_reviews_data_ops = random.sample(yelp_reviews_data, num_lines_to_select)
        # Send data to Kafka in batches of 10 records with a 10-second sleep time between batches
        batch_size = 10
        for i in range(0, len(yelp_reviews_data_ops), batch_size):
            batch = yelp_reviews_data_ops[i:i + batch_size]
            iteration = (i // batch_size) + 1
            send_to_kafka(producer, topic, batch, iteration)
            print('Producer sleeps for 10 seconds:')
            time.sleep(10)

    except Exception as e:
        print(f"Error: {e}")
    finally:
        producer.close()

def kafka_consumer_call(topic):
    """
    Establishes a Kafka consumer, starts consuming messages in a separate thread, and handles interruption gracefully.

    Args:
        topic (str): Kafka topic from which messages will be consumed.
    """
    print('Consumer receving data:')

    # Create a Kafka consumer
    consumer = KafkaConsumer(topic, bootstrap_servers='localhost:9092', auto_offset_reset='earliest')

    # Start consuming messages in a separate thread
    consumer_thread = threading.Thread(target=consume_messages, args=(consumer,))
    consumer_thread.start()

    try:
        # Main thread sleeps for 30 seconds to allow message consumption
        time.sleep(30)
    except KeyboardInterrupt:
        print("Received keyboard interrupt. Stopping consumer gracefully.")
    finally:
        # Set a flag to stop the consumer thread gracefully
        consumer.close()
        consumer_thread.join()

    print("Exiting the main thread.")



### **Kafka Producer** & **Kafka Consumer**

### Sequential Operation

**Calling Kafka Producer**

In [47]:
kafka_producer_call(drive_link,topic)

Producer sending data:
yelp data length : 10
b'{"review_id": "VDn0TqsdpxbNUBZEXkakUQ", "user_id": "16_EZvrDyxjDgxq6OUdbhQ", "business_id": "WaRZsQEcEHwRk0T2dJ-cuA", "stars": 5.0, "useful": 1, "funny": 0, "cool": 0, "text": "Fabulous! Best happy hour, fresh menu, Delish- drinks, if I had the time, I\'d come here every day!", "date": "2014-02-24 03:20:20"}'
b'{"review_id": "MSZuaItS46OO_47XmktjcQ", "user_id": "BO1xi6vjg46qcwIfJDGIAg", "business_id": "kGNzqiYKDns-IU80uHF4sw", "stars": 3.0, "useful": 0, "funny": 0, "cool": 0, "text": "I had chicken BBQ burritos with all kinds of stuff that I like. I love the it ! Good place to sit down with family & had a meal. I wish they had a place for kids to play that would be even better !", "date": "2014-09-25 15:08:18"}'
b'{"review_id": "cIY-1dh_AggHAseMnMQkng", "user_id": "d8z61FRYZzvEqjWNqS8Xlw", "business_id": "ICqgjbOpBD9SUtE5PQC9sA", "stars": 4.0, "useful": 0, "funny": 0, "cool": 0, "text": "A definite go-to place if you\'re in Treasure Island

**Calling Kafka Consumer**

In [48]:
kafka_consumer_call(topic)

Consumer receving data:
b'{"review_id": "VDn0TqsdpxbNUBZEXkakUQ", "user_id": "16_EZvrDyxjDgxq6OUdbhQ", "business_id": "WaRZsQEcEHwRk0T2dJ-cuA", "stars": 5.0, "useful": 1, "funny": 0, "cool": 0, "text": "Fabulous! Best happy hour, fresh menu, Delish- drinks, if I had the time, I\'d come here every day!", "date": "2014-02-24 03:20:20"}'
b'{"review_id": "MSZuaItS46OO_47XmktjcQ", "user_id": "BO1xi6vjg46qcwIfJDGIAg", "business_id": "kGNzqiYKDns-IU80uHF4sw", "stars": 3.0, "useful": 0, "funny": 0, "cool": 0, "text": "I had chicken BBQ burritos with all kinds of stuff that I like. I love the it ! Good place to sit down with family & had a meal. I wish they had a place for kids to play that would be even better !", "date": "2014-09-25 15:08:18"}'
b'{"review_id": "cIY-1dh_AggHAseMnMQkng", "user_id": "d8z61FRYZzvEqjWNqS8Xlw", "business_id": "ICqgjbOpBD9SUtE5PQC9sA", "stars": 4.0, "useful": 0, "funny": 0, "cool": 0, "text": "A definite go-to place if you\'re in Treasure Island! Love the char-grill

Streaming output truncated to the last 5000 lines.


### Parallalization of Producer and Consumer Call

### Kafka Producer

In [10]:
def kafka_producer_parallal_call():
    """
    This function reads records from a Yelp dataset file and sends them as messages to a Kafka topic.

    It uses a Kafka producer to parallelly send records to the specified Kafka topic. The records are read
    from a JSON dataset file, and a sleep interval is used to manage the sending rate.

    Returns:
    None
    """
    kafka_topic = "yelp_reviews"
    dataset_link = '/content/drive/MyDrive/CapstoneProjectData/yelp_academic_dataset_review.json'

    # Create a Kafka producer instance
    producer = KafkaProducer(
        bootstrap_servers="localhost:9092",
        value_serializer=lambda x: dumps(x).encode('utf-8'))

    # Open the Yelp dataset file for reading
    with open(dataset_link, 'r') as dataset:
        record_count = 0
        print('Sending the first batch of records')

        # Iterate through the dataset
        for record_line in dataset:
            # Sleep every 10 records (excluding the first) to manage the sending rate
            if record_count % 10 == 0 and record_count > 0:
                print('Sending the next batch of records')
                time.sleep(10)

            # Break the loop after processing a certain number of records
            if record_count == 101:
                break

            # Increment the record counter
            record_count += 1

            # Load the current record from JSON
            current_record = json.loads(record_line)
            print(current_record)

            # Send the current record to the Kafka topic
            producer.send(kafka_topic, current_record)

            # Flush the producer to ensure data is sent
            producer.flush()

        # Close the Yelp dataset file after processing
        dataset.close()

# Call the Kafka producer function
kafka_producer_parallal_call()

Sending the first batch of records
{'review_id': 'KU_O5udG6zpxOg-VcAEodg', 'user_id': 'mh_-eMZ6K5RLWhZyISBhwA', 'business_id': 'XQfwVwDr-v0ZS3_CbbE5Xw', 'stars': 3.0, 'useful': 0, 'funny': 0, 'cool': 0, 'text': "If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", 'date': '2018-07-07 22:09:11'}
{'review_id': 'BiTunyQ73aT9WBnpR9DZGw', 'user_id': 'OyoGAe7OKpv6SyGZT5g77Q', 'business_id': '7ATYjTIgM3jUlt4UM3IypQ', 'stars': 5.0, 'useful': 1, 'funny': 0, 'cool': 1, 'text': "I've taken a lot of spin classes over the years, and nothing c

#### Kafka Consumer

In [12]:
def kafka_consumer_parallal_call():
    """
    This function sets up a Kafka consumer to read messages from a specified Kafka topic.

    The consumer is configured with the provided Kafka consumer group, topic, and server details.
    It consumes messages from the topic using the KafkaConsumer and processes each message
    by printing its contents. A sleep interval is used to manage the processing rate.

    Returns:
    None
    """
    # Set the Kafka consumer group, topic, and server details
    consumer_group_name = "academia_cons_group"
    topic_name = "yelp_reviews"
    server = 'localhost:9092'
    print("Starting Kafka Consumer Application...")

    try:
        # Create a Kafka consumer
        kafka_consumer = KafkaConsumer(
            topic_name,
            bootstrap_servers=server,
            auto_offset_reset='latest',  # Start consuming from the latest offset
            enable_auto_commit=True,     # Enable automatic committing of offsets
            value_deserializer=lambda x: loads(x.decode('utf-8'))
        )

        # Iterate through messages from the Kafka topic
        for cur_message in kafka_consumer:
            print("\n=============================")
            print("Received Message:")
            print(cur_message.value)
            print("===============================")

            # Sleep to control processing rate
            time.sleep(5)

    except Exception as ex:
        print("Failure occur during reading the message from Kafka")

# Call the Kafka consumer function
# kafka_consumer_parallal_call()

#### Parallal Processing

In [13]:
# Start the producer process
kafka_producer_process = Process(target=kafka_producer_parallal_call)
kafka_producer_process.start()
# Start the consumer process
kafka_consumer_process = Process(target=kafka_consumer_parallal_call)
kafka_consumer_process.start()
# Wait for the producer process to complete
kafka_producer_process.join()

Starting Kafka Consumer Application...
Sending the first batch of records


{'review_id': 'KU_O5udG6zpxOg-VcAEodg', 'user_id': 'mh_-eMZ6K5RLWhZyISBhwA', 'business_id': 'XQfwVwDr-v0ZS3_CbbE5Xw', 'stars': 3.0, 'useful': 0, 'funny': 0, 'cool': 0, 'text': "If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", 'date': '2018-07-07 22:09:11'}
{'review_id': 'BiTunyQ73aT9WBnpR9DZGw', 'user_id': 'OyoGAe7OKpv6SyGZT5g77Q', 'business_id': '7ATYjTIgM3jUlt4UM3IypQ', 'stars': 5.0, 'useful': 1, 'funny': 0, 'cool': 1, 'text': "I've taken a lot of spin classes over the years, and nothing compares to the classes at Body Cycl

**Part 2: Pyspark Streaming**

> Consider a scenario where you are a data engineer working for a ride-sharing company called "PyRides." The company has a fleet of drivers who continuously send GPS location data and trip information during their shifts. As a data engineer, you process this real-time streaming data and gain insights into driver performance.

> **The data stream contains the following fields:**

        1.   **driver_id:** The unique identifier of the driver.
        2.   **timestamp:** The timestamp of the GPS location or trip event.
        3.   **latitude:** The latitude of the driver's location.
        4.   **longitude:** The longitude of the driver's location.
        5.   **trip_distance:** The distance covered by the driver during the trip (if it's a trip event).

> **Task 1: Count of Unique Drivers**
Your task is to implement a PySpark Streaming code to calculate the count of unique drivers within a sliding window of 10 minutes, updated every 5 minutes. Display the results for each window update.

> **Task 2: Average Trip Duration**
Your task is to implement a PySpark Streaming code to calculate the average trip duration for each driver within a tumbling window of 15 minutes. Display the results for each window update.

> **Task 3: Idle Time Detection**
Your task is to implement a PySpark Streaming code to detect idle time for each driver using session windows. Consider it an idle session if the driver's location remains unchanged for more than 30 minutes. Display the start and end timestamps of each idle session.

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=26360be6c1dc2794a54e62ef87ac597f159d79908de309180d386553086e5f9d
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


**Importing Libraries**

In [4]:
import os
import warnings
import threading
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.functions import * # window, col, countDistinct, avg, lag, when, approx_count_distinct, sum, min, max,from_unixtime, unix_timestamp,expr
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, LongType
from pyspark.sql.window import Window
from pyspark.streaming import StreamingContext
from pyspark import SparkConf, SparkContext

**Mounting Google Drive**

In [5]:
# Check if Google Drive is already mounted
if not os.path.exists('/content/drive'):
    # Mount Google Drive if it's not already mounted
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

Mounted at /content/drive


**Common Variable Declaration**

In [6]:
# Create a SparkSession
spark_session = SparkSession.builder.master("local").appName("PyRidesDriverPerformance").config('spark.ui.port', '4050').getOrCreate()
# File path to the JSON data
file_path = '/content/drive/MyDrive/CapstoneProjectData/'
# Define the sliding window duration and slide duration
window_duration_part1, slide_duration_part1 = "10 minutes", "5 minutes"
# Define the tumbling window of 15 minutes
window_duration_part2 = "15 minutes"
# Define the session window of 30 minutes
session_gap_duration = "30 minutes"
# Define a threshold for idle session detection (30 minutes in seconds)
idle_threshold_seconds = 1800
# Define the schema for the streaming data
schema = StructType([
    StructField("driver_id", StringType(), nullable=False),
    StructField("timestamp", TimestampType(), nullable=False),
    StructField("latitude", DoubleType(), nullable=False),
    StructField("longitude", DoubleType(), nullable=False),
    StructField("trip_distance", DoubleType(), nullable=True),
    StructField("event_type", StringType(), nullable=True)  # Set nullable=True if trip_distance can be missing
])

**Loading the streaming data**

In [7]:
# Read the JSON file into a DataFrame using the specified schema.
ride_information_stream = spark_session.readStream.format("json").option('multiline', True).schema(schema).json(file_path)
# Display the schema of the DataFrame.
ride_information_stream.printSchema()
# Select all columns from the streaming DataFrame.
ride_information_dataframe = ride_information_stream.select("*")
# Start a streaming query to write the DataFrame into a memory sink.The query is named "ride_information_using_stream" and operates in append mode.The data is processed every 5 seconds as specified by the 'trigger' parameter.
ride_information_query = ride_information_dataframe.writeStream.format("memory").outputMode("append").queryName("ride_information_using_stream").trigger(processingTime='5 seconds').start()

root
 |-- driver_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- event_type: string (nullable = true)



In [8]:
# Retrieve data from the "ride_information_using_stream" temporary view using SQL query
ride_information = spark_session.sql("select * from ride_information_using_stream")

In [9]:
# Display the data
ride_information.show()

+---------+-------------------+------------------+-------------------+------------------+----------+
|driver_id|          timestamp|          latitude|          longitude|     trip_distance|event_type|
+---------+-------------------+------------------+-------------------+------------------+----------+
|     null|               null|              null|               null|              null|      null|
|     D001|2023-07-30 05:55:48|37.538849579360246|-121.22748988885533| 3.423076573047031|      Trip|
|     D001|2023-07-30 07:10:08| 37.93476356947273| -121.0215678805398| 5.072880200441588|      Trip|
|     D001|2023-07-30 07:11:01|37.795197133875064|  -121.920222434928|              null|       GPS|
|     D001|2023-07-30 05:20:21| 37.46871545044007|-121.05888928225791|1.5685949947600415|      Trip|
|     D001|2023-07-30 05:27:29| 37.67930257604861| -121.7174686646489|              null|       GPS|
|     D001|2023-07-30 06:20:49| 37.57858135955766|-121.45726820764473|              null|  

**Task 1: Count of Unique Drivers**

In [10]:
# Sorting the ride information DataFrame by 'driver_id' and 'timestamp'
ride_information = ride_information.orderBy('driver_id', 'timestamp')
# Grouping the ride information DataFrame by time windows and calculating the count of unique drivers within each window
unique_drivers_count = ride_information.groupBy(
    window(col("timestamp"), window_duration_part1, slide_duration_part1)
).agg(countDistinct("driver_id").alias("unique_drivers_count"))

In [11]:
# Print a descriptive message indicating the task being performed
print('Task 1: Count of Unique Drivers : ')
# Perform ordering of the DataFrame containing the count of unique drivers using the 'window' column.
ordered_unique_drivers_data = unique_drivers_count.orderBy('window')
# Display the ordered DataFrame containing the count of unique drivers
ordered_unique_drivers_data.show(ordered_unique_drivers_data.count(), False)

Task 1: Count of Unique Drivers : 
+------------------------------------------+--------------------+
|window                                    |unique_drivers_count|
+------------------------------------------+--------------------+
|{2023-07-30 05:05:00, 2023-07-30 05:15:00}|47                  |
|{2023-07-30 05:10:00, 2023-07-30 05:20:00}|50                  |
|{2023-07-30 05:15:00, 2023-07-30 05:25:00}|50                  |
|{2023-07-30 05:20:00, 2023-07-30 05:30:00}|50                  |
|{2023-07-30 05:25:00, 2023-07-30 05:35:00}|50                  |
|{2023-07-30 05:30:00, 2023-07-30 05:40:00}|50                  |
|{2023-07-30 05:35:00, 2023-07-30 05:45:00}|50                  |
|{2023-07-30 05:40:00, 2023-07-30 05:50:00}|50                  |
|{2023-07-30 05:45:00, 2023-07-30 05:55:00}|49                  |
|{2023-07-30 05:50:00, 2023-07-30 06:00:00}|50                  |
|{2023-07-30 05:55:00, 2023-07-30 06:05:00}|50                  |
|{2023-07-30 06:00:00, 2023-07-30 06:10:0

**Task 2: Average Trip Duration**

In [12]:
# Selecting specific columns "driver_id", "timestamp", and "event_type" from the ride_information DataFrame
eventwise_ride = ride_information.select("driver_id", "timestamp", "event_type")
# Dropping rows where all columns have null values
eventwise_ride = eventwise_ride.dropna(how="all")

In [18]:
# Define a Window specification that partitions the data by 'driver_id' and orders it by 'timestamp'
windows_specs = Window.partitionBy('driver_id').orderBy('timestamp')
# Add a new column 'prev_timestamp' that holds the timestamp of the previous event for the same driver
eventwise_data_within_window = eventwise_ride.withColumn("prev_timestamp", lag('timestamp').over(windows_specs))
# Calculate the duration between the current event's timestamp and the previous event's timestamp
eventwise_data_within_window = eventwise_data_within_window.withColumn(
    "event_duration", (col("timestamp").cast('long') - col('prev_timestamp').cast('long'))
)

In [19]:
# Display the DataFrame containing trip data within the specified window
eventwise_data_within_window.show()

+---------+-------------------+----------+-------------------+--------------+
|driver_id|          timestamp|event_type|     prev_timestamp|event_duration|
+---------+-------------------+----------+-------------------+--------------+
|     D001|2023-07-30 05:12:56|      Trip|               null|          null|
|     D001|2023-07-30 05:15:57|       GPS|2023-07-30 05:12:56|           181|
|     D001|2023-07-30 05:16:08|      Trip|2023-07-30 05:15:57|            11|
|     D001|2023-07-30 05:17:46|      Trip|2023-07-30 05:16:08|            98|
|     D001|2023-07-30 05:17:51|      Trip|2023-07-30 05:17:46|             5|
|     D001|2023-07-30 05:20:21|      Trip|2023-07-30 05:17:51|           150|
|     D001|2023-07-30 05:20:57|      Trip|2023-07-30 05:20:21|            36|
|     D001|2023-07-30 05:21:07|       GPS|2023-07-30 05:20:57|            10|
|     D001|2023-07-30 05:23:03|      Trip|2023-07-30 05:21:07|           116|
|     D001|2023-07-30 05:23:27|       GPS|2023-07-30 05:23:03|  

In [20]:
# Define a window specification for ordering by driver_id
window_spec_idle_duration = Window.orderBy(col("driver_id"))
# Calculate the actual event duration by lagging the event_duration column by -1 (previous row)
eventwise_data_within_window = eventwise_data_within_window.withColumn(
    "event_duration_actual", lag(col("event_duration"), -1).over(window_spec_idle_duration)
)
# Drop the unused column
eventwise_data_within_window = eventwise_data_within_window.drop('event_duration')
# Show the DataFrame with the calculated actual event duration
eventwise_data_within_window.show()

+---------+-------------------+----------+-------------------+---------------------+
|driver_id|          timestamp|event_type|     prev_timestamp|event_duration_actual|
+---------+-------------------+----------+-------------------+---------------------+
|     D001|2023-07-30 05:12:56|      Trip|               null|                  181|
|     D001|2023-07-30 05:15:57|       GPS|2023-07-30 05:12:56|                   11|
|     D001|2023-07-30 05:16:08|      Trip|2023-07-30 05:15:57|                   98|
|     D001|2023-07-30 05:17:46|      Trip|2023-07-30 05:16:08|                    5|
|     D001|2023-07-30 05:17:51|      Trip|2023-07-30 05:17:46|                  150|
|     D001|2023-07-30 05:20:21|      Trip|2023-07-30 05:17:51|                   36|
|     D001|2023-07-30 05:20:57|      Trip|2023-07-30 05:20:21|                   10|
|     D001|2023-07-30 05:21:07|       GPS|2023-07-30 05:20:57|                  116|
|     D001|2023-07-30 05:23:03|      Trip|2023-07-30 05:21:07|   

In [21]:
# Filtering the eventwise_ride DataFrame to keep only rows where the "event_type" column is "Trip"
only_trip_data = eventwise_data_within_window.filter(eventwise_ride.event_type == "Trip")
# Displaying the resulting DataFrame with only "Trip" event type data
only_trip_data.show()

+---------+-------------------+----------+-------------------+---------------------+
|driver_id|          timestamp|event_type|     prev_timestamp|event_duration_actual|
+---------+-------------------+----------+-------------------+---------------------+
|     D001|2023-07-30 05:12:56|      Trip|               null|                  181|
|     D001|2023-07-30 05:16:08|      Trip|2023-07-30 05:15:57|                   98|
|     D001|2023-07-30 05:17:46|      Trip|2023-07-30 05:16:08|                    5|
|     D001|2023-07-30 05:17:51|      Trip|2023-07-30 05:17:46|                  150|
|     D001|2023-07-30 05:20:21|      Trip|2023-07-30 05:17:51|                   36|
|     D001|2023-07-30 05:20:57|      Trip|2023-07-30 05:20:21|                   10|
|     D001|2023-07-30 05:23:03|      Trip|2023-07-30 05:21:07|                   24|
|     D001|2023-07-30 05:29:41|      Trip|2023-07-30 05:27:29|                   15|
|     D001|2023-07-30 05:29:56|      Trip|2023-07-30 05:29:41|   

In [22]:
# Calculate the average trip duration within the specified window for each driver
average_trip_duration = only_trip_data.groupBy('driver_id', window('timestamp', window_duration_part2)).agg(round(avg('event_duration_actual')/60,2).alias('average trip duration'))
# Filter out rows where the calculated average trip duration is not null
average_trip_duration = average_trip_duration.filter(col('average trip duration').isNotNull())

In [23]:
# Display the calculated average trip duration along with the count of rows in the DataFrame.
average_trip_duration.show(average_trip_duration.count(), truncate=False)

+---------+------------------------------------------+---------------------+
|driver_id|window                                    |average trip duration|
+---------+------------------------------------------+---------------------+
|D001     |{2023-07-30 05:00:00, 2023-07-30 05:15:00}|3.02                 |
|D001     |{2023-07-30 05:15:00, 2023-07-30 05:30:00}|0.94                 |
|D001     |{2023-07-30 05:30:00, 2023-07-30 05:45:00}|0.9                  |
|D001     |{2023-07-30 05:45:00, 2023-07-30 06:00:00}|0.65                 |
|D001     |{2023-07-30 06:00:00, 2023-07-30 06:15:00}|1.09                 |
|D001     |{2023-07-30 06:15:00, 2023-07-30 06:30:00}|1.53                 |
|D001     |{2023-07-30 06:30:00, 2023-07-30 06:45:00}|1.18                 |
|D001     |{2023-07-30 06:45:00, 2023-07-30 07:00:00}|1.04                 |
|D001     |{2023-07-30 07:00:00, 2023-07-30 07:15:00}|1.24                 |
|D002     |{2023-07-30 05:00:00, 2023-07-30 05:15:00}|0.43                 |

**Task 3: Idle Time Detection**

In [24]:
# Select specific columns "driver_id", "timestamp", and "event_type" from the ride_information DataFrame.
eventwise_ride = ride_information.select("driver_id", "timestamp", "event_type")
# Drop rows where all columns have missing (null) values.
eventwise_ride = eventwise_ride.dropna(how="all")
# Display the resulting DataFrame "eventwise_ride" to inspect the data.
eventwise_ride.show()

+---------+-------------------+----------+
|driver_id|          timestamp|event_type|
+---------+-------------------+----------+
|     D001|2023-07-30 05:12:56|      Trip|
|     D001|2023-07-30 05:15:57|       GPS|
|     D001|2023-07-30 05:16:08|      Trip|
|     D001|2023-07-30 05:17:46|      Trip|
|     D001|2023-07-30 05:17:51|      Trip|
|     D001|2023-07-30 05:20:21|      Trip|
|     D001|2023-07-30 05:20:57|      Trip|
|     D001|2023-07-30 05:21:07|       GPS|
|     D001|2023-07-30 05:23:03|      Trip|
|     D001|2023-07-30 05:23:27|       GPS|
|     D001|2023-07-30 05:27:29|       GPS|
|     D001|2023-07-30 05:29:41|      Trip|
|     D001|2023-07-30 05:29:56|      Trip|
|     D001|2023-07-30 05:31:50|       GPS|
|     D001|2023-07-30 05:32:08|       GPS|
|     D001|2023-07-30 05:32:20|      Trip|
|     D001|2023-07-30 05:32:34|      Trip|
|     D001|2023-07-30 05:34:54|      Trip|
|     D001|2023-07-30 05:36:21|       GPS|
|     D001|2023-07-30 05:38:09|       GPS|
+---------+

In [25]:
# Define a window specification for partitioning the data by driver_id and ordering by timestamp.
windows_specs = Window.partitionBy('driver_id').orderBy('timestamp')
# Create a new DataFrame 'eventwise_ride_within_window' by adding a new column 'prev_timestamp' which contains the timestamp of the previous row within the same partition.
eventwise_ride_within_window = eventwise_ride.withColumn("prev_timestamp", lag('timestamp').over(windows_specs))

In [26]:
# Display the contents of the DataFrame "eventwise_ride_within_window"
eventwise_ride_within_window.show()

+---------+-------------------+----------+-------------------+
|driver_id|          timestamp|event_type|     prev_timestamp|
+---------+-------------------+----------+-------------------+
|     D001|2023-07-30 05:12:56|      Trip|               null|
|     D001|2023-07-30 05:15:57|       GPS|2023-07-30 05:12:56|
|     D001|2023-07-30 05:16:08|      Trip|2023-07-30 05:15:57|
|     D001|2023-07-30 05:17:46|      Trip|2023-07-30 05:16:08|
|     D001|2023-07-30 05:17:51|      Trip|2023-07-30 05:17:46|
|     D001|2023-07-30 05:20:21|      Trip|2023-07-30 05:17:51|
|     D001|2023-07-30 05:20:57|      Trip|2023-07-30 05:20:21|
|     D001|2023-07-30 05:21:07|       GPS|2023-07-30 05:20:57|
|     D001|2023-07-30 05:23:03|      Trip|2023-07-30 05:21:07|
|     D001|2023-07-30 05:23:27|       GPS|2023-07-30 05:23:03|
|     D001|2023-07-30 05:27:29|       GPS|2023-07-30 05:23:27|
|     D001|2023-07-30 05:29:41|      Trip|2023-07-30 05:27:29|
|     D001|2023-07-30 05:29:56|      Trip|2023-07-30 05

In [27]:
# Calculate the duration of each event within the specified window
eventwise_ride_details_with_idle_duration = eventwise_ride_within_window.withColumn(
    "event_duration",
    (col("timestamp").cast('long') - col('prev_timestamp').cast('long'))
)

In [28]:
# Display the DataFrame containing ride details along with calculated idle durations
eventwise_ride_details_with_idle_duration.show()

+---------+-------------------+----------+-------------------+--------------+
|driver_id|          timestamp|event_type|     prev_timestamp|event_duration|
+---------+-------------------+----------+-------------------+--------------+
|     D001|2023-07-30 05:12:56|      Trip|               null|          null|
|     D001|2023-07-30 05:15:57|       GPS|2023-07-30 05:12:56|           181|
|     D001|2023-07-30 05:16:08|      Trip|2023-07-30 05:15:57|            11|
|     D001|2023-07-30 05:17:46|      Trip|2023-07-30 05:16:08|            98|
|     D001|2023-07-30 05:17:51|      Trip|2023-07-30 05:17:46|             5|
|     D001|2023-07-30 05:20:21|      Trip|2023-07-30 05:17:51|           150|
|     D001|2023-07-30 05:20:57|      Trip|2023-07-30 05:20:21|            36|
|     D001|2023-07-30 05:21:07|       GPS|2023-07-30 05:20:57|            10|
|     D001|2023-07-30 05:23:03|      Trip|2023-07-30 05:21:07|           116|
|     D001|2023-07-30 05:23:27|       GPS|2023-07-30 05:23:03|  

In [29]:
# Create a window specification for ordering by the "driver_id" column
window_spec_idle_duration = Window.orderBy(col("driver_id"))
# Add a new column "event_duration_actual" to the DataFrame
# It calculates the lag of the "event_duration" column with an offset of -1 using the window specification defined above
# This effectively moves the "event_duration" value one row up in the DataFrame for each driver
eventwise_ride_details_with_idle_duration_new = eventwise_ride_details_with_idle_duration.withColumn(
    "event_duration_actual", lag(col("event_duration"), -1).over(window_spec_idle_duration)
)
# Drop the unused column
eventwise_ride_details_with_idle_duration_new = eventwise_ride_details_with_idle_duration_new.drop('event_duration')
# Display the resulting DataFrame with the added column
# The "count()" function is used to show all rows in the DataFrame without truncation

eventwise_ride_details_with_idle_duration_new.show(eventwise_ride_details_with_idle_duration_new.count(), truncate=False)

+---------+-------------------+----------+-------------------+---------------------+
|driver_id|timestamp          |event_type|prev_timestamp     |event_duration_actual|
+---------+-------------------+----------+-------------------+---------------------+
|D001     |2023-07-30 05:12:56|Trip      |null               |181                  |
|D001     |2023-07-30 05:15:57|GPS       |2023-07-30 05:12:56|11                   |
|D001     |2023-07-30 05:16:08|Trip      |2023-07-30 05:15:57|98                   |
|D001     |2023-07-30 05:17:46|Trip      |2023-07-30 05:16:08|5                    |
|D001     |2023-07-30 05:17:51|Trip      |2023-07-30 05:17:46|150                  |
|D001     |2023-07-30 05:20:21|Trip      |2023-07-30 05:17:51|36                   |
|D001     |2023-07-30 05:20:57|Trip      |2023-07-30 05:20:21|10                   |
|D001     |2023-07-30 05:21:07|GPS       |2023-07-30 05:20:57|116                  |
|D001     |2023-07-30 05:23:03|Trip      |2023-07-30 05:21:07|24 

In [32]:
# Filter the DataFrame to retain only rows with the "GPS" event type
only_GPS_data = eventwise_ride_details_with_idle_duration_new.filter(eventwise_ride_details_with_idle_duration_new.event_type == "GPS")
# Display the content of the filtered DataFrame along with its total row count
only_GPS_data.show(only_GPS_data.count())

+---------+-------------------+----------+-------------------+---------------------+
|driver_id|          timestamp|event_type|     prev_timestamp|event_duration_actual|
+---------+-------------------+----------+-------------------+---------------------+
|     D001|2023-07-30 05:15:57|       GPS|2023-07-30 05:12:56|                   11|
|     D001|2023-07-30 05:21:07|       GPS|2023-07-30 05:20:57|                  116|
|     D001|2023-07-30 05:23:27|       GPS|2023-07-30 05:23:03|                  242|
|     D001|2023-07-30 05:27:29|       GPS|2023-07-30 05:23:27|                  132|
|     D001|2023-07-30 05:31:50|       GPS|2023-07-30 05:29:56|                   18|
|     D001|2023-07-30 05:32:08|       GPS|2023-07-30 05:31:50|                   12|
|     D001|2023-07-30 05:36:21|       GPS|2023-07-30 05:34:54|                  108|
|     D001|2023-07-30 05:38:09|       GPS|2023-07-30 05:36:21|                   44|
|     D001|2023-07-30 05:38:53|       GPS|2023-07-30 05:38:09|   

In [33]:
# Identify idle sessions
idle_sessions = only_GPS_data.withColumn("Idle Duration Flag", when(col("event_duration_actual") > idle_threshold_seconds, 1).otherwise(0))
idle_sessions.show()

+---------+-------------------+----------+-------------------+---------------------+------------------+
|driver_id|          timestamp|event_type|     prev_timestamp|event_duration_actual|Idle Duration Flag|
+---------+-------------------+----------+-------------------+---------------------+------------------+
|     D001|2023-07-30 05:15:57|       GPS|2023-07-30 05:12:56|                   11|                 0|
|     D001|2023-07-30 05:21:07|       GPS|2023-07-30 05:20:57|                  116|                 0|
|     D001|2023-07-30 05:23:27|       GPS|2023-07-30 05:23:03|                  242|                 0|
|     D001|2023-07-30 05:27:29|       GPS|2023-07-30 05:23:27|                  132|                 0|
|     D001|2023-07-30 05:31:50|       GPS|2023-07-30 05:29:56|                   18|                 0|
|     D001|2023-07-30 05:32:08|       GPS|2023-07-30 05:31:50|                   12|                 0|
|     D001|2023-07-30 05:36:21|       GPS|2023-07-30 05:34:54|  

In [34]:
# Print a descriptive message indicating the purpose of the following code block
print('Driver idle for 30 minutes')
# Filter the DataFrame 'idle_sessions' to retain rows where the "Idle Duration Flag" column is greater than 0,This filters out rows representing periods of driver activity and keeps only rows corresponding to idle sessions
idle_sessions = idle_sessions.filter(col("Idle Duration Flag") > 0)
# Display the filtered DataFrame 'idle_sessions' using the 'show' method
idle_sessions.show(idle_sessions.count(), truncate=False)

Driver idle for 30 minutes
+---------+---------+----------+--------------+---------------------+------------------+
|driver_id|timestamp|event_type|prev_timestamp|event_duration_actual|Idle Duration Flag|
+---------+---------+----------+--------------+---------------------+------------------+
+---------+---------+----------+--------------+---------------------+------------------+



#### Without using session windows

In [35]:
# Group the DataFrame 'only_GPS_data' by the 'driver_id' column, Calculate the maximum value of the 'event_duration_actual' column for each driver group, Convert the maximum duration from seconds to minutes using the round function with 2 decimal places
# Rename the calculated column to 'Max Idle Duration' using the alias function
driver_wise_max_idle_time = only_GPS_data.groupBy(col('driver_id')).agg(round(max(col('event_duration_actual'))/60,2).alias('Max Idle Duration'))
# Display the results of the 'driver_wise_max_idle_time' DataFrame
driver_wise_max_idle_time.show(driver_wise_max_idle_time.count())

+---------+-----------------+
|driver_id|Max Idle Duration|
+---------+-----------------+
|     D001|              6.7|
|     D002|             4.97|
|     D003|             6.37|
|     D004|             6.17|
|     D005|             4.02|
|     D006|             3.17|
|     D007|             4.65|
|     D008|             5.73|
|     D009|             5.88|
|     D010|             7.17|
|     D011|             4.32|
|     D012|             4.13|
|     D013|              5.4|
|     D014|             3.48|
|     D015|             4.82|
|     D016|              6.8|
|     D017|             4.43|
|     D018|             6.57|
|     D019|             5.53|
|     D020|             4.78|
|     D021|              4.8|
|     D022|              5.5|
|     D023|             6.27|
|     D024|             5.27|
|     D025|             4.72|
|     D026|             4.12|
|     D027|              4.3|
|     D028|              4.7|
|     D029|             4.98|
|     D030|             4.58|
|     D031

#### Using session window of 30 minutes (as nothing specific has been mention)

In [36]:
# Group the DataFrame 'only_GPS_data' by the 'driver_id' column, Calculate the maximum value of the 'event_duration_actual' column for each driver group, Convert the maximum duration from seconds to minutes using the round function with 2 decimal places
# Rename the calculated column to 'Max Idle Duration' using the alias function
driver_wise_max_idle_time = only_GPS_data.groupBy(col('driver_id'),session_window("timestamp", "30 minutes")).agg(round(max(col('event_duration_actual'))/60,2).alias('Max Idle Duration'))
# Display the results of the 'driver_wise_max_idle_time' DataFrame
driver_wise_max_idle_time.show(driver_wise_max_idle_time.count())

+---------+--------------------+-----------------+
|driver_id|      session_window|Max Idle Duration|
+---------+--------------------+-----------------+
|     D001|{2023-07-30 05:15...|              6.7|
|     D002|{2023-07-30 05:12...|             4.97|
|     D003|{2023-07-30 05:14...|             6.37|
|     D004|{2023-07-30 05:13...|             6.17|
|     D005|{2023-07-30 05:12...|             4.02|
|     D006|{2023-07-30 05:14...|             3.17|
|     D007|{2023-07-30 05:12...|             4.65|
|     D008|{2023-07-30 05:12...|             5.73|
|     D009|{2023-07-30 05:16...|             5.88|
|     D010|{2023-07-30 05:13...|             7.17|
|     D011|{2023-07-30 05:16...|             4.32|
|     D012|{2023-07-30 05:11...|             4.13|
|     D013|{2023-07-30 05:13...|              5.4|
|     D014|{2023-07-30 05:12...|             3.48|
|     D015|{2023-07-30 05:14...|             4.82|
|     D016|{2023-07-30 05:11...|              6.8|
|     D017|{2023-07-30 05:18...

#### Using both session window and without it, we are getting same result.